In [ ]:
import requests
import json
import os
import pandas
import util

In [ ]:
TOKEN = ""
BEACON_QUERY_URL = "https://beacon-wod.maris.nl/api/query"

In [ ]:
query_parameters = []

# TIME
query_parameters.append(util.column("time"))
query_parameters.append(
    util.function_call("to_timestamp_nanos", ["time"], "COMMON_TIME")
)
# LONGITUDE
query_parameters.append(util.column("lon"))
query_parameters.append(util.column("lon.units"))
# COMMON LONGITUDE
query_parameters.append(util.column("lon", "COMMON_LONGITUDE"))
query_parameters.append(util.value("degrees_east", "COMMON_LONGITUDE_UNITS"))
query_parameters.append(util.value("Longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
query_parameters.append(util.value("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))

# LATITUDE
query_parameters.append(util.column("lat"))
query_parameters.append(util.column("lat.units"))
# COMMON LATITUDE
query_parameters.append(util.column("lat", "COMMON_LATITUDE"))
query_parameters.append(util.value("degrees_north", "COMMON_LATITUDE_UNITS"))
query_parameters.append(util.value("Latitude", "COMMON_LATITUDE_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
query_parameters.append(util.value("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))

# DEPTH
query_parameters.append(util.column("z"))
query_parameters.append(util.column("z.units"))
query_parameters.append(util.column("z_WODflag"))

# COMMON DEPTH
query_parameters.append(util.column("z", "COMMON_ORIGIN_DEPTH"))
query_parameters.append(
    util.function_call("map_wod_quality_flag", ["z_WODflag"], "COMMON_ORIGIN_DEPTH_QC")
)
query_parameters.append(util.column("z.units", "COMMON_ORIGIN_DEPTH_UNITS"))
query_parameters.append(util.value("Depth", "COMMON_ORIGIN_DEPTH_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ADEPZZ01", "COMMON_ORIGIN_DEPTH_P01"))
query_parameters.append(util.value("SDN:P06::ULAA", "COMMON_ORIGIN_DEPTH_P06"))

query_parameters.append(util.column("z", "COMMON_DEPTH"))
query_parameters.append(
    util.function_call("map_wod_quality_flag", ["z_WODflag"], "COMMON_DEPTH_QC")
)
query_parameters.append(util.value("m", "COMMON_DEPTH_UNITS"))
query_parameters.append(util.value("Depth", "COMMON_DEPTH_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
query_parameters.append(util.value("SDN:P06::ULAA", "COMMON_DEPTH_P06"))

In [ ]:
# DATASET METADATA
query_parameters.append(util.value("", "COMMON_EDMERP_CODE"))
query_parameters.append(util.value("", "COMMON_EDMO_CODE"))
query_parameters.append(util.function_call("map_wod_platform_c17", ["Platform"], alias="COMMON_PLATFORM_C17"))

# COMMON_ODV TAG
query_parameters.append(
    util.function_call(
        "concat", [util.value("BEACON_WOD"), "@identifier"], "COMMON_ODV_TAG"
    )
)

# BEACON SOURCE REFERENCE
query_parameters.append(util.value("BEACON_WOD", "SOURCE_BDI"))
query_parameters.append(util.column("@identifier", "SOURCE_BDI_DATASET_ID"))

# ORIGIN METADATA FORWARDING
metadata = [
    "dataset",
    ".institution",
    "Platform",
    "country",
    "WOD_cruise_identifier",
    "wod_unique_cast",
]
for m in metadata:
    query_parameters.append(util.column(m))

In [ ]:
# MAP CHLOROPHYLL
query_parameters.append(util.column("Chlorophyll"))
query_parameters.append(util.column("Chlorophyll_WODflag"))
query_parameters.append(util.column("Chlorophyll.units"))

query_parameters.append(util.column("Chlorophyll", "COMMON_ORIGIN_CHLOROPHYLL"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Chlorophyll_WODflag"], "COMMON_ORIGIN_CHLOROPHYLL_QC"
    )
)
query_parameters.append(
    util.column("Chlorophyll.standard_name", "COMMON_ORIGIN_CHLOROPHYLL_STANDARD_NAME")
)
query_parameters.append(
    util.column("Chlorophyll.units", "COMMON_ORIGIN_CHLOROPHYLL_UNITS")
)
query_parameters.append(
    util.value("SDN:P01::CHLTVOLU", "COMMON_ORIGIN_CHLOROPHYLL_P01")
)
query_parameters.append(util.value("SDN:P06::UMMC", "COMMON_ORIGIN_CHLOROPHYLL_P06"))

# MAP COMMON CHLOROPHYLL
query_parameters.append(util.column("Chlorophyll", "COMMON_CHLOROPHYLL_PER_VOLUME"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Chlorophyll_WODflag"], "COMMON_CHLOROPHYLL_PER_VOLUME_QC"
    )
)
query_parameters.append(
    util.value(
        "mass_concentration_of_chlorophyll_in_sea_water",
        "COMMON_CHLOROPHYLL_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("mg m-3", "COMMON_CHLOROPHYLL_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::CHLTVOLU", "COMMON_CHLOROPHYLL_PER_VOLUME_P01")
)
query_parameters.append(
    util.value("SDN:P06::UMMC", "COMMON_CHLOROPHYLL_PER_VOLUME_P06")
)
# MAP COMMON CHLOROPHYLL INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l05", ["Chlorophyll_Instrument"], "COMMON_CHLOROPHYLL_L05"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l22", ["Chlorophyll_Instrument"], "COMMON_CHLOROPHYLL_L22"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l33", ["Chlorophyll_Instrument"], "COMMON_CHLOROPHYLL_L33"
    )
)
query_parameters.append(util.column("Chlorophyll_Instrument"))

In [ ]:
# MAP Oxygen
query_parameters.append(util.column("Oxygen"))
query_parameters.append(util.column("Oxygen_WODflag"))
query_parameters.append(util.column("Oxygen.units"))

query_parameters.append(util.column("Oxygen", "COMMON_ORIGIN_OXYGEN"))
query_parameters.append(
    util.function_call("map_wod_quality_flag", ["Oxygen_WODflag"], "COMMON_ORIGIN_OXYGEN_QC")
)
query_parameters.append(
    util.column("Oxygen.standard_name", "COMMON_ORIGIN_OXYGEN_STANDARD_NAME")
)
query_parameters.append(util.column("Oxygen.units", "COMMON_ORIGIN_OXYGEN_UNITS"))
query_parameters.append(util.value("SDN:P01::DOXMZZXX", "COMMON_ORIGIN_OXYGEN_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_ORIGIN_OXYGEN_P06"))

# MAP COMMON Oxygen
query_parameters.append(
    util.function_call(
        "map_units_seadatanet",
        [util.value("SDN:P06::KGUM"), util.value("SDN:P06::UPOX"), "Oxygen"],
        "COMMON_OXYGEN_PER_VOLUME",
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Oxygen_WODflag"], "COMMON_OXYGEN_PER_VOLUME_QC"
    )
)
query_parameters.append(
    util.value(
        "mole_concentration_of_dissolved_molecular_oxygen_in_sea_water",
        "COMMON_OXYGEN_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_OXYGEN_PER_VOLUME_UNITS"))
query_parameters.append(util.value("SDN:P01::DOXYZZXX", "COMMON_OXYGEN_PER_VOLUME_P01"))
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_OXYGEN_PER_VOLUME_P06"))

# MAP COMMON Oxygen MASS
query_parameters.append(util.column("Oxygen", "COMMON_OXYGEN_PER_MASS"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Oxygen_WODflag"], "COMMON_OXYGEN_PER_MASS_QC"
    )
)
query_parameters.append(
    util.value(
        "moles_of_oxygen_per_unit_mass_in_sea_water",
        "COMMON_OXYGEN_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_OXYGEN_PER_MASS_UNITS"))
query_parameters.append(util.value("SDN:P01::DOXMZZXX", "COMMON_OXYGEN_PER_MASS_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_OXYGEN_PER_MASS_P06"))

# MAP COMMON Oxygen INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l05", ["Oxygen_Instrument"], "COMMON_OXYGEN_L05"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l22", ["Oxygen_Instrument"], "COMMON_OXYGEN_L22"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l33", ["Oxygen_Instrument"], "COMMON_OXYGEN_L33"
    )
)
query_parameters.append(util.column("Oxygen_Instrument"))

In [ ]:
# MAP Phosphate
query_parameters.append(util.column("Phosphate"))
query_parameters.append(util.column("Phosphate_WODflag"))
query_parameters.append(util.column("Phosphate.units"))

query_parameters.append(util.column("Phosphate", "COMMON_ORIGIN_PHOSPHATE"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Phosphate_WODflag"], "COMMON_ORIGIN_PHOSPHATE_QC"
    )
)
query_parameters.append(
    util.column("Phosphate.standard_name", "COMMON_ORIGIN_PHOSPHATE_STANDARD_NAME")
)
query_parameters.append(util.column("Phosphate.units", "COMMON_ORIGIN_PHOSPHATE_UNITS"))
query_parameters.append(util.value("SDN:P01::MDMAP906", "COMMON_ORIGIN_PHOSPHATE_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_ORIGIN_PHOSPHATE_P06"))

# MAP COMMON Phosphate
query_parameters.append(
    util.function_call(
        "map_units_seadatanet",
        [util.value("SDN:P06::KGUM"), util.value("SDN:P06::UPOX"), "Phosphate"],
        "COMMON_PHOSPHATE_PER_VOLUME",
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Phosphate_WODflag"], "COMMON_PHOSPHATE_PER_VOLUME_QC"
    )
)
query_parameters.append(
    util.value(
        "mole_concentration_of_phosphate_in_sea_water",
        "COMMON_PHOSPHATE_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_PHOSPHATE_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::PHOSZZXX", "COMMON_PHOSPHATE_PER_VOLUME_P01")
)
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_PHOSPHATE_PER_VOLUME_P06"))

# MAP COMMON Phosphate MASS
query_parameters.append(util.column("Phosphate", "COMMON_PHOSPHATE_PER_MASS"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Phosphate_WODflag"], "COMMON_PHOSPHATE_PER_MASS_QC"
    )
)
query_parameters.append(
    util.value(
        "moles_of_phosphate_per_unit_mass_in_sea_water",
        "COMMON_PHOSPHATE_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_PHOSPHATE_PER_MASS_UNITS"))
query_parameters.append(
    util.value("SDN:P01::MDMAP906", "COMMON_PHOSPHATE_PER_MASS_P01")
)
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_PHOSPHATE_PER_MASS_P06"))

# MAP COMMON Phosphate INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l05",
        ["Phosphate_Instrument"],
        "COMMON_PHOSPHATE_L05",
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l22",
        ["Phosphate_Instrument"],
        "COMMON_PHOSPHATE_L22",
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l33",
        ["Phosphate_Instrument"],
        "COMMON_PHOSPHATE_L33",
    )
)
query_parameters.append(util.column("Phosphate_Instrument"))

In [ ]:
# MAP Silicate
query_parameters.append(util.column("Silicate"))
query_parameters.append(util.column("Silicate_WODflag"))
query_parameters.append(util.column("Silicate.units"))

query_parameters.append(util.column("Silicate", "COMMON_ORIGIN_SILICATE"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Silicate_WODflag"], "COMMON_ORIGIN_SILICATE_QC"
    )
)
query_parameters.append(
    util.column("Silicate.standard_name", "COMMON_ORIGIN_SILICATE_STANDARD_NAME")
)
query_parameters.append(util.column("Silicate.units", "COMMON_ORIGIN_SILICATE_UNITS"))
query_parameters.append(util.value("SDN:P01::MDMAP012", "COMMON_ORIGIN_SILICATE_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_ORIGIN_SILICATE_P06"))

# MAP COMMON Silicate
query_parameters.append(
    util.function_call(
        "map_units_seadatanet",
        [util.value("SDN:P06::KGUM"), util.value("SDN:P06::UPOX"), "Silicate"],
        "COMMON_SILICATE_PER_VOLUME",
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Silicate_WODflag"], "COMMON_SILICATE_PER_VOLUME_QC"
    )
)
query_parameters.append(
    util.value(
        "mole_concentration_of_silicate_in_sea_water",
        "COMMON_SILICATE_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol l-1", "COMMON_SILICATE_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::SLCAZZXX", "COMMON_SILICATE_PER_VOLUME_P01")
)
query_parameters.append(util.value("SDN:P06::UPOX", "COMMON_SILICATE_PER_VOLUME_P06"))

# MAP COMMON Silicate MASS
query_parameters.append(util.column("Silicate", "COMMON_SILICATE_PER_MASS"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Silicate_WODflag"], "COMMON_SILICATE_PER_MASS_QC"
    )
)
query_parameters.append(
    util.value(
        "moles_of_silicate_per_unit_mass_in_sea_water",
        "COMMON_SILICATE_PER_MASS_STANDARD_NAME",
    )
)
query_parameters.append(util.value("umol kg-1", "COMMON_SILICATE_PER_MASS_UNITS"))
query_parameters.append(util.value("SDN:P01::MDMAP012", "COMMON_SILICATE_PER_MASS_P01"))
query_parameters.append(util.value("SDN:P06::KGUM", "COMMON_SILICATE_PER_MASS_P06"))

# MAP COMMON Silicate INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l05", ["Silicate_Instrument"], "COMMON_SILICATE_L05"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l22", ["Silicate_Instrument"], "COMMON_SILICATE_L22"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l33", ["Silicate_Instrument"], "COMMON_SILICATE_L33"
    )
)
query_parameters.append(util.column("Silicate_Instrument"))

In [ ]:
# MAP Salinity
query_parameters.append(util.column("Salinity"))
query_parameters.append(util.column("Salinity_WODflag"))

query_parameters.append(util.column("Salinity", "COMMON_ORIGIN_SALINITY"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Salinity_WODflag"], "COMMON_ORIGIN_SALINITY_QC"
    )
)
query_parameters.append(
    util.column("Salinity.standard_name", "COMMON_ORIGIN_SALINITY_STANDARD_NAME")
)
query_parameters.append(util.value("", "COMMON_ORIGIN_SALINITY_UNITS"))
query_parameters.append(util.value("SDN:P01::PSLTZZ01", "COMMON_ORIGIN_SALINITY_P01"))
query_parameters.append(util.value("SDN:P06::UUUU", "COMMON_ORIGIN_SALINITY_P06"))

# MAP COMMON Salinity
query_parameters.append(util.column("Salinity", "COMMON_SALINITY"))
query_parameters.append(
    util.function_call("map_wod_quality_flag", ["Salinity_WODflag"], "COMMON_SALINITY_QC")
)
query_parameters.append(
    util.value(
        "sea_water_salinity",
        "COMMON_SALINITY_STANDARD_NAME",
    )
)
query_parameters.append(util.value("1e-3", "COMMON_SALINITY_UNITS"))
query_parameters.append(util.value("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
query_parameters.append(util.value("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
# MAP COMMON Salinity INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l05", ["Salinity_Instrument"], "COMMON_SALINITY_L05"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l22", ["Salinity_Instrument"], "COMMON_SALINITY_L22"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l33", ["Salinity_Instrument"], "COMMON_SALINITY_L33"
    )
)
query_parameters.append(util.column("Salinity_Instrument"))

In [ ]:
# MAP Temperature
query_parameters.append(util.column("Temperature"))
query_parameters.append(util.column("Temperature_WODflag"))

query_parameters.append(util.column("Temperature", "COMMON_ORIGIN_TEMPERATURE"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Temperature_WODflag"], "COMMON_ORIGIN_TEMPERATURE_QC"
    )
)
query_parameters.append(
    util.column("Temperature.standard_name", "COMMON_ORIGIN_TEMPERATURE_STANDARD_NAME")
)
query_parameters.append(
    util.value("Temperature.units", "COMMON_ORIGIN_TEMPERATURE_UNITS")
)
query_parameters.append(
    util.value("SDN:P01::TEMPPR01", "COMMON_ORIGIN_TEMPERATURE_P01")
)
query_parameters.append(util.value("SDN:P06::UPAA", "COMMON_ORIGIN_TEMPERATURE_P06"))

# MAP COMMON Temperature
query_parameters.append(util.column("Temperature", "COMMON_TEMPERATURE"))
query_parameters.append(
    util.function_call(
        "map_wod_quality_flag", ["Temperature_WODflag"], "COMMON_TEMPERATURE_QC"
    )
)
query_parameters.append(
    util.value(
        "sea_water_temperature",
        "COMMON_TEMPERATURE_STANDARD_NAME",
    )
)
query_parameters.append(util.value("degree_C", "COMMON_TEMPERATURE_UNITS"))
query_parameters.append(util.value("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
query_parameters.append(util.value("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
# MAP COMMON Temperature INSTRUMENTS
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l05", ["Temperature_Instrument"], "COMMON_TEMPERATURE_L05"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l22", ["Temperature_Instrument"], "COMMON_TEMPERATURE_L22"
    )
)
query_parameters.append(
    util.function_call(
        "map_wod_instrument_l33", ["Temperature_Instrument"], "COMMON_TEMPERATURE_L33"
    )
)
query_parameters.append(util.column("Temperature_Instrument"))

In [ ]:
def build_query(start_time, end_time) -> dict:
    return {
        "select": query_parameters,
        "filters": [
            {
                "column": "time",
                "min": start_time,
                "max": end_time,
            },
            {
                "or": [
                    {
                        "is_not_null": {
                            "column": "Chlorophyll",
                        }
                    },
                    {
                        "is_not_null": {
                            "column": "Oxygen",
                        }
                    },
                    {
                        "is_not_null": {
                            "column": "Phosphate",
                        }
                    },
                    {
                        "is_not_null": {
                            "column": "Silicate",
                        }
                    },
                ]
            },
        ],
        "output": {"format": "parquet"},
    }

In [ ]:
os.makedirs("data", exist_ok=True)

for year in range(2000, 2024):
    start_time = f"{year}-01-01T00:00:00.000"
    end_time = f"{year}-12-31T23:59:59.999"
    print(build_query(start_time, end_time))
    print(f"Downloading {start_time} - {end_time}")
    # Download data
    with requests.post(
        BEACON_QUERY_URL,
        json=build_query(start_time, end_time),
        headers={"Authorization": f"Bearer {TOKEN}"},
        stream=True,
    ) as response:
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            print(response.text)
            exit(1)
        response.raise_for_status()
        with open(f"data/WOD_{year}.parquet", "wb") as f:
            for chunk in response.iter_content(chunk_size=4096 * 128):
                if chunk:
                    f.write(chunk)

In [ ]:
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile("data/WOD_2000.parquet")

# Read each row group as a pandas DataFrame
print(parquet_file.num_row_groups)
table = parquet_file.read_row_group(1)
df = table.to_pandas()
df